In [2]:
import pandas as pd
from wmfdata import hive
from wmfdata.utils import sig_figs
from numbers import Number

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


In [6]:
%matplotlib inline

In [7]:
def fmt_num(x):
    if isinstance(x, Number):
        x = sig_figs(x, 3)
        
        if x < 5:
            return pct(x)
        else:
            return "{:,.0f}".format(x)
    else:
        return x

def pct(n):
    return "{:.1%}".format(n)

def find_latest(ser):
    nn_ser = ser[~ser.isnull()]
    return nn_ser.index[-1].date()

def calc_rpt(ser):
    nn_ser = ser[~ser.isnull()]
    cur_mo = nn_ser.index[-1].date()
    cur = nn_ser.iloc[-1]
    mo_prev = nn_ser.iloc[-2]
    
    try:
        yr_prev = nn_ser.iloc[-13]
        yoy_change = (cur / yr_prev) - 1
    except IndexError:
        yoy_change = None
    
    mom_change = (cur / mo_prev) - 1
    
    res = [cur_mo, cur, mom_change, yoy_change]
    return pd.Series(
        [fmt_num(n) for n in res],
        index=["latest_month", "value", "mom_change", "yoy_change"]
    )

In [8]:
mob_wikis = (
    "hiwiki",
    "bnwiki",
    "idwiki",
    "arwiki",
    "mrwiki",
    "fawiki",
    "swwiki",
    "tlwiki",
    "zhwikiquote",
    "thwiki",
    "arzwiki",
    "mlwiki",
    "tawiki",
    "knwiki",
    "ptwiktionary",
    "azwiki",
    "guwiki",
    "kywiki",
    "sqwiki",
    "mswiki"
)

gn_countries = (
    "AD", "AL", "AT", "AX", "BA", "BE", "BG", "CH", "CY", "CZ",
    "DE", "DK", "EE", "ES", "FI", "FO", "FR", "FX", "GB", "GG",
    "GI", "GL", "GR", "HR", "HU", "IE", "IL", "IM", "IS", "IT",
    "JE", "LI", "LU", "LV", "MC", "MD", "ME", "MK", "MT", "NL",
    "NO", "PL", "PT", "RO", "RS", "RU", "SE", "SI", "SJ", "SK",
    "SM", "TR", "VA", "AU", "CA", "HK", "MO", "NZ", "JP", "SG",
    "KR", "TW", "US", 
    # Have to include unlocated edits—otherwise they'll be counted as Global South
    "--"
)

# Global South countries

In [43]:
# Earliest data still available in geoeditors_daily
hive.run("""
select
    min(date) as min_date,
    max(date) as max_date
from wmf.geoeditors_daily
where
    month >= "2001-01"
""")

,min_date,max_date
0,2018-09-01,2018-10-31


## Edits and editors

In [5]:
gs_activity_r = hive.run(
"""
with gs_editors as (
    select 
        month,
        user_fingerprint_or_id as user_id,
        sum(edit_count) as edit_count,
        sum(namespace_zero_edit_count) as namespace_zero_edit_count,
        -- Treat the user as a bot if it matches on any wiki
        max(is_bot_by_name or array_contains(user_groups, "bot")) as bot
    from wmf.geoeditors_daily gd
    left join canonical_data.countries cdc
    on gd.country_code = cdc.iso_code
    left join wmf.mediawiki_user_history muh
    on
        gd.wiki_db = muh.wiki_db and
        gd.user_fingerprint_or_id = muh.user_id and
        muh.snapshot = "{mediawiki_history_snapshot}" and
        muh.end_timestamp is null
    where
        month = "{metrics_month}" and
        economic_region = "Global South" and
        not user_is_anonymous
    group by month, user_fingerprint_or_id
)
select
    month,
    sum(edit_count) as total_edits,
    sum(if(not bot, edit_count, 0)) as nonbot_edits,
    sum(cast(namespace_zero_edit_count >= 5 as int)) as active_editors
from gs_editors
group by month
""".format(
    mediawiki_history_snapshot="2018-10",
    metrics_month="2018-10"
))

gs_activity_r.tail()

,month,total_edits,nonbot_edits,active_editors
0,2018-10,1979771,1979769,19891


In [12]:
(gs_activity_r
.assign(month=lambda df: pd.to_datetime(df["month"]))
.set_index("month")
.apply(calc_rpt)
.transpose()
)

,latest_month,value,mom_change,yoy_change
total_edits,2018-10-01,"1,980,000",-4.3%,None
nonbot_edits,2018-10-01,"1,980,000",-4.3%,None
active_editors,2018-10-01,"19,900",-1.1%,None


## Edits and editors, old method

In [131]:
old_gn_countries = (
    "AD", "AL", "AT", "AX", "BA", "BE", "BG", "CH", "CY", "CZ",
    "DE", "DK", "EE", "ES", "FI", "FO", "FR", "FX", "GB", "GG",
    "GI", "GL", "GR", "HR", "HU", "IE", "IL", "IM", "IS", "IT",
    "JE", "LI", "LU", "LV", "MC", "MD", "ME", "MK", "MT", "NL",
    "NO", "PL", "PT", "RO", "RS", "RU", "SE", "SI", "SJ", "SK",
    "SM", "TR", "VA", "AU", "CA", "HK", "MO", "NZ", "JP", "SG",
    "KR", "TW", "US", 
)

In [132]:
old_gs_edits = wmf.hive.run("""
with gs_editors as (
    select 
        wiki_db,
        user_fingerprint_or_id as user_id
    from wmf.geoeditors_daily
    where
        month >= "{start}" and
        country_code not in {gn_countries} and
        user_is_anonymous = 0
    group by wiki_db, user_fingerprint_or_id
)
select
    date_format(event_timestamp, "yyyy-MM-01") as month,
    count(*) as total_edits,
    sum(cast(
        !(event_user_is_bot_by_name or array_contains(event_user_groups, "bot")
    ) as int)) as nonbot_edits
from gs_editors ge
left join wmf.mediawiki_history mh
on
    ge.wiki_db = mh.wiki_db and
    ge.user_id = event_user_id and
    snapshot = "{snapshot}"
where
    event_entity = "revision" and
    event_type = "create" and
    event_timestamp >= "{start}"
group by date_format(event_timestamp, "yyyy-MM-01")
""".format(
    gn_countries=repr(old_gn_countries),
    snapshot="2018-09",
    start="2018-04"
))

In [133]:
old_gs_edits

,month,total_edits,nonbot_edits
0,2018-04-01,7576888,7576753
1,2018-05-01,8592750,8592464
2,2018-06-01,8093110,8092757
3,2018-07-01,8366023,8365815
4,2018-08-01,9596925,9596533
5,2018-09-01,9047193,9047065


In [ ]:
old_gs_editors = wmf.hive.run("""
with gs_editors as (
    select 
        wiki_db,
        user_fingerprint_or_id as user_id
    from wmf.geoeditors_daily
    where
        month >= "{start}" and
        country_code not in {gn_countries} and
        user_is_anonymous = 0
    group by wiki_db, user_fingerprint_or_id
)
select
    month,
    sum(cast(content_edits >= 5 as int)) as active_editors
from (
    select
        date_format(event_timestamp, "yyyy-MM-01") as month,
        count(*) as content_edits
    from gs_editors ge
    left join wmf.mediawiki_history mh
    on
        ge.wiki_db = mh.wiki_db and
        ge.user_id = event_user_id and
        snapshot = "{snapshot}"
    where
        event_entity = "revision" and
        event_type = "create" and
        event_timestamp >= "{start}" and
        page_namespace_is_content = 1 and
        !(event_user_is_bot_by_name or array_contains(event_user_groups, "bot"))
    group by event_user_text, date_format(event_timestamp, "yyyy-MM-01")
) combined_eds
group by month
""".format(
    gn_countries=repr(old_gn_countries),
    snapshot="2018-09",
    start="2018-04"
))

In [137]:
old_gs_editors

,month,active_editors
0,2018-04-01,13980
1,2018-05-01,14713
2,2018-06-01,15724
3,2018-07-01,22466
4,2018-08-01,22254
5,2018-09-01,22964


## New editor retention

In [ ]:
gs_ner = wmf.hive.run([
"SET hive.auto.convert.join=false",
"""
with gs_new_edits as (
    select 
        gd.wiki_db,
        event_user_text as user_name,
        event_timestamp as edit_dt,
        event_user_creation_timestamp as registration_dt
    from wmf.geoeditors_daily gd
    left join canonical_data.countries cdc
    on gd.country_code = cdc.iso_code
    left join wmf.mediawiki_history mh
    on
        gd.wiki_db = mh.wiki_db and
        gd.user_fingerprint_or_id = mh.user_id and
        mh.snapshot = "{snapshot}"
    where
        month >= "{start}" and
        economic_region = "Global South" and
        not event_user_is_anonymous and
        not event_user_is_created_by_system and
        not (event_user_is_bot_by_name or array_contains(event_user_groups, "bot")) and
        event_user_creation_timestamp >= "{start}" and
        event_user_creation_timestamp < "{end}"
)
select 
    1st_month.cohort as cohort,
    sum(cast(1st_month.edits >= 1 as int)) as new_editors,
    sum(cast(2nd_month.edits >= 1 as int)) / sum(cast(1st_month.edits >= 1 as int)) as new_editor_retention
from (
    select
        user_name,
        wiki_db as wiki,
        substr(registration_dt, 0, 7) as cohort,
        count(*) as edits
    from gs_new_edits
    where
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") <
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60))
    group by user_name, registration_dt, wiki_db
    ) 1st_month
left join (
    select
        user_name,
        wiki_db as wiki,
        substr(registration_dt, 0, 7) as cohort,
        count(*) as edits
    from gs_new_edits
    where
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") >=
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60)) and
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") <
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (60*24*60*60))
        group by user_name, registration_dt, wiki_db
    ) 2nd_month
on
    (1st_month.user_name = 2nd_month.user_name and
    1st_month.wiki = 2nd_month.wiki and
    1st_month.cohort = 2nd_month.cohort)
group by 1st_month.cohort
""".format(
    snapshot="2018-10",
    start="2018-05",
    end="2018-09"
)])

gs_ner

In [15]:
gs_ner

,cohort,new_editors,new_editor_retention
0,2018-05,2108,0.008065
1,2018-06,2747,0.012741
2,2018-07,4627,0.009942
3,2018-08,37810,0.001455


## New editor retention, old method

In [8]:
old_gs_ner = wmf.hive.run("""
with gs_edits as (
    select 
        gd.wiki_db,
        event_user_text as user_name,
        event_timestamp as edit_dt,
        event_user_creation_timestamp as registration_dt
    from wmf.geoeditors_daily gd
    left join wmf.mediawiki_history mh
    on
        gd.wiki_db = mh.wiki_db and
        gd.user_fingerprint_or_id = event_user_id and
        snapshot = "{snapshot}"
    where
        month >= "{start}" and
        country_code not in {gn_countries} and
        gd.user_is_anonymous = 0 and
        event_entity = "revision" and
        event_type = "create" and
        event_timestamp >= "{start}" and
        event_user_is_created_by_system = 0 and
        event_user_creation_timestamp >= "{start}" and
        event_user_creation_timestamp < "{end}" and
        not (event_user_is_bot_by_name or array_contains(event_user_groups, "bot"))
)
select 
    1st_month.cohort,
    sum(cast(1st_month.edits >= 1 as int)) as new_editors,
    sum(cast(2nd_month.edits >= 1 as int)) / sum(cast(1st_month.edits >= 1 as int)) as new_editor_retention
from (
    select
        user_name,
        wiki_db as wiki,
        substr(registration_dt, 0, 7) as cohort,
        count(*) as edits
    from gs_edits
    where
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") <
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60))
    group by user_name, registration_dt, wiki_db
    ) 1st_month
left join (
    select
        user_name,
        wiki_db as wiki,
        substr(registration_dt, 0, 7) as cohort,
        count(*) as edits
    from gs_edits
    where
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") >=
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60)) and
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") <
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (60*24*60*60))
        group by user_name, registration_dt, wiki_db
    ) 2nd_month
on
    (1st_month.user_name = 2nd_month.user_name and
    1st_month.wiki = 2nd_month.wiki and
    1st_month.cohort = 2nd_month.cohort)
group by 1st_month.cohort
""".format(
    gn_countries=repr(old_gn_countries),
    snapshot="2018-09",
    start="2018-04",
    end="2018-10"
))

In [9]:
old_gs_ner

,1st_month.cohort,new_editors,new_editor_retention
0,2018-04,1580,0.414557
1,2018-05,1977,0.516945
2,2018-06,4001,0.409648
3,2018-07,35658,0.048320
4,2018-08,37474,0.032156
5,2018-09,38596,NaN


# Mobile-heavy wikis

In [ ]:
mh_edits = wmf.hive.run([
"SET hive.auto.convert.join=false",
"""
select
    date_format(event_timestamp, "yyyy-MM-01") as month,
    count(*) as total_edits,
    sum(cast(not (event_user_is_bot_by_name or array_contains(event_user_groups, "bot")) as int)) as nonbot_edits
from wmf.mediawiki_history
left join canonical_data.mobile_heavy_wikis mhw
on wiki_db = database_code
where
    snapshot = "{snapshot}" and
    event_entity = "revision" and
    event_type = "create" and
    mhw.database_code is not null and
    event_timestamp >= "{start}" and
    event_timestamp < "{end}" and
    not event_user_is_anonymous
group by date_format(event_timestamp, "yyyy-MM-01")
""".format(
    snapshot="2018-10",
    start="2017-06",
    end="2018-11"
)])

In [24]:
(mh_edits
.assign(month=lambda df: pd.to_datetime(df["month"]))
.set_index("month")
.apply(calc_rpt)
.transpose()
)

,latest_month,value,mom_change,yoy_change
total_edits,2018-10-01,"958,000",13.1%,-8.4%
nonbot_edits,2018-10-01,"438,000",-1.0%,6.2%


In [26]:
mh_editors = wmf.hive.run([
"SET hive.auto.convert.join=false",
"""
select
    month,
    sum(cast(content_edits >= 5 as int)) as active_editors
from (
    select
        date_format(event_timestamp, "yyyy-MM-01") as month,
        count(*) as content_edits
    from wmf.mediawiki_history
    left join canonical_data.mobile_heavy_wikis mhw
    on wiki_db = database_code
    where
        snapshot = "{snapshot}" and
        event_entity = "revision" and
        event_type = "create" and
        mhw.database_code is not null and
        event_timestamp >= "{start}" and
        event_timestamp < "{end}" and
        not event_user_is_anonymous and
        not (event_user_is_bot_by_name or array_contains(event_user_groups, "bot")) and
        page_namespace_is_content
    group by date_format(event_timestamp, "yyyy-MM-01"), event_user_text
) editors
group by month
""".format(
    snapshot="2018-10",
    start="2017-10",
    end="2018-11"
)])

In [29]:
(mh_editors
.sort_values("month")
.assign(month=lambda df: pd.to_datetime(df["month"]))
.set_index("month")
.apply(calc_rpt)
.transpose()
)

,latest_month,value,mom_change,yoy_change
active_editors,2018-10-01,"3,800",-1.2%,13.7%


In [ ]:
mh_ner = wmf.hive.run([
"SET hive.auto.convert.join=false",
"""
with mh_edits as (
    select 
        wiki_db,
        event_user_text as user_name,
        event_timestamp as edit_dt,
        event_user_creation_timestamp as registration_dt
    from wmf.mediawiki_history
    left join canonical_data.mobile_heavy_wikis mhw
    on wiki_db = database_code
    where
        snapshot = "{snapshot}" and
        event_entity = "revision" and
        event_type = "create" and
        event_timestamp >= "{start}" and
        mhw.database_code is not null and
        not event_user_is_anonymous and
        not event_user_is_created_by_system and
        event_user_creation_timestamp >= "{start}" and
        event_user_creation_timestamp < "{end}" and
        not (event_user_is_bot_by_name or array_contains(event_user_groups, "bot"))
)
select 
    1st_month.cohort as cohort,
    sum(cast(1st_month.edits >= 1 as int)) as new_editors,
    sum(cast(2nd_month.edits >= 1 as int)) / sum(cast(1st_month.edits >= 1 as int)) as new_editor_retention
from (
    select
        user_name,
        wiki_db as wiki,
        substr(registration_dt, 0, 7) as cohort,
        count(*) as edits
    from mh_edits
    where
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") <
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60))
    group by user_name, registration_dt, wiki_db
    ) 1st_month
left join (
    select
        user_name,
        wiki_db as wiki,
        substr(registration_dt, 0, 7) as cohort,
        count(*) as edits
    from mh_edits
    where
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") >=
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (30*24*60*60)) and
        unix_timestamp(edit_dt, "yyyy-MM-dd HH:mm:ss.0") <
            (unix_timestamp(registration_dt, "yyyy-MM-dd HH:mm:ss.0") + (60*24*60*60))
        group by user_name, registration_dt, wiki_db
    ) 2nd_month
on
    (1st_month.user_name = 2nd_month.user_name and
    1st_month.wiki = 2nd_month.wiki and
    1st_month.cohort = 2nd_month.cohort)
group by 1st_month.cohort
""".format(
    snapshot="2018-10",
    start="2017-08",
    end="2018-09"
)])

mh_ner.tail()

In [39]:
mh_ner.tail()

,cohort,new_editors,new_editor_retention
8,2018-04,8504,0.040569
9,2018-05,7836,0.041092
10,2018-06,7164,0.047041
11,2018-07,7943,0.050736
12,2018-08,8541,0.050580


In [40]:
(mh_ner
.assign(cohort=lambda df: pd.to_datetime(df["cohort"]))
.set_index("cohort")
.apply(calc_rpt)
.transpose()
)

,latest_month,value,mom_change,yoy_change
new_editors,2018-08-01,"8,540",7.5%,12.1%
new_editor_retention,2018-08-01,5.1%,-0.3%,19.3%
